In [2]:
import json

In [1]:
category = 'Florida'

In [3]:
from tqdm import tqdm
from llama import Tokenizer
from llama.tokenizer import ChatFormat

max_seq_len = 1024

tokenizer = Tokenizer(
                model_path='/data1/home/xingmei/llama3-demo/llama3/Meta-Llama-3-8B-Instruct/tokenizer.model',                                                                                            
            )
formatter = ChatFormat(tokenizer)

review = json.load(open(f'/data1/home/xingmei/GRE/data/GoogleLocalData/Food/extracted_review/{category}.json'))
            
dialogs = []
gmap_ids = []
for gmap_id, texts in tqdm(review.items()):

    dialog = [
        # {
        #     "role": "system",
        #     "content": "You are an excellent review summarizer.",
        # },
        {   "role": "user", 
            "content": """Given the user reviews of a restaurant, craft a concise summary that captures the restaurant's characteristics.

[User Reviews]
"""         },
    ] 
    token_left = max_seq_len - len(formatter.encode_dialog_prompt(dialog))

    gmap_ids.append(gmap_id)
    prompt = ''
    for i, text in enumerate(texts):
        token_len = len(tokenizer.encode(text, bos=True, eos=True))
        token_left -= token_len
        if token_left < 0:
            break
        prompt += f"{i + 1}. {text}\n"
    dialog[-1]['content'] += prompt
    assert len(formatter.encode_dialog_prompt(dialog)) <= max_seq_len + 100, len(formatter.encode_dialog_prompt(dialog))

    dialogs.append(dialog)

100%|██████████| 54603/54603 [02:21<00:00, 385.86it/s]


In [4]:
with open(f'/data1/home/xingmei/GRE/data/GoogleLocalData/Food/extracted_review/prompt_{category}.json', 'w') as f:
    json.dump([dialogs, gmap_ids], f)

In [5]:
len(gmap_ids)

54603

In [6]:
dialogs[0]

[{'role': 'user',
  'content': "Given the user reviews of a restaurant, craft a concise summary that captures the restaurant's characteristics.\n\n[User Reviews]\n1. Great small place I did know peruvian cusine was so good. I will go back\n"}]

In [7]:
len(formatter.encode_dialog_prompt(dialogs[0]))

53

In [8]:
print(dialogs[0][-1]['content'])

Given the user reviews of a restaurant, craft a concise summary that captures the restaurant's characteristics.

[User Reviews]
1. Great small place I did know peruvian cusine was so good. I will go back



In [9]:
review[gmap_ids[0]]

['Great small place I did know peruvian cusine was so good. I will go back']